# Nuclei Segmentation

Let's begin with nuclei segmentation.
We can either segment the DAPI channel and then filter out the healthy nuclei, segment only the healthy nuclei or go for the ISL1 channel that has the label for the nuclei of interest.

Let's select an image and then crop out a small piece of it to make testing easier.

In [ ]:
from pathlib import Path

import numpy as np
from czifile import CziFile
from napari import Viewer
from scipy.ndimage import distance_transform_edt
from skimage.morphology import disk, opening

In [ ]:
DATA_DIR = Path(
    "../data/20240729/980/20x_with_z_correction/NS#4_Healthy_5uM_MI132_D19_G3BP,Iselt,DAPI_20x+ZCorr.czi"
)
image_handle = CziFile(DATA_DIR)
img = np.squeeze(image_handle.asarray())[2, :, 1250:2000, 1250:2000]

In [ ]:
scale = {
    values_dict["Id"]: values_dict["Value"] * 10**6
    for values_dict in image_handle.metadata(raw=False)["ImageDocument"]["Metadata"][
        "Scaling"
    ]["Items"]["Distance"]
}
spacing = (scale["Z"], scale["Y"], scale["X"])

## Histogram Equalization

We should first try some histogram equalization as the intensity values ar every different across planes.
I am not sure this will solve some of the saturation problems though.

In [ ]:
from skimage.exposure import equalize_adapthist  # noqa

In [ ]:
kernel = (50 / scale["Z"], 50 / scale["Y"], 50 / scale["X"])

equalized_img = equalize_adapthist(
    img,
    kernel_size=kernel,
    clip_limit=0.05,
)

In [ ]:
viewer = Viewer()
viewer.add_image(img, scale=spacing, visible=False)
viewer.add_image(equalized_img, scale=spacing, visible=True)

## Cellpose

Let's begin by trying out cellpose.

In [ ]:
from cellpose import models  # noqa
from cellpose.io import logger_setup  # noqa

logger_setup()

In [ ]:
model = models.Cellpose(model_type="nuclei")

After a couple attempts with several algorithms, I thought there might be issues with all the saturated nuclei.
To circunvent this, I made a mask for those pixels, calculated the distance transform and added this to the original image in order to have some values here.

In [ ]:
anisotropy = scale["Z"] / scale["X"]
print(anisotropy)

In [ ]:
diameter = 10 / scale["X"]
print(diameter)

In [ ]:
masks, flows, styles, diams = model.eval(
    [equalized_img],
    batch_size=8,
    diameter=diameter,
    z_axis=0,
    do_3D=True,
    channels=[0, 0],
    # stitch_threshold=0.5,
    normalize=False,
    anisotropy=anisotropy,
)

In [ ]:
viewer = Viewer()
viewer.add_image(img, scale=spacing)
viewer.add_labels(masks[0], scale=spacing)

It's working quite well for some nuclei.
Some are still over segmented and some others are not segmented.

In [ ]:
this_disk = disk(3)
disk_3d = np.zeros((3,) + this_disk.shape)
disk_3d[1] = this_disk
polished_mask = opening(masks[0], disk_3d)

In [ ]:
viewer = Viewer()
viewer.add_image(img, scale=spacing)
viewer.add_labels(masks[0], scale=spacing, visible=False)
viewer.add_labels(polished_mask, scale=spacing)

After this smoothing of the labels, segmentation of nuclei looks quite fine.

## StarDist

Maybe stardist might work.
Not tested yet.

## Clesperanto Voronoi Otsu

Maybe Clesperanto's Voronoi and Otsu labelling works well here.

In [ ]:
import pyclesperanto_prototype as cle  # noqa

cle.available_device_names()

In [ ]:
input_gpu = cle.push(img[3])
input_gpu

I was unable to make clesperanto work on the laptop.

## Thresholding

Let's go for typical thresholding techniques as nuclei might be separated enough to not need watershed or maybe a simple watershed.

In [ ]:
from skimage.feature import peak_local_max  # noqa
from skimage.filters import median, scharr, threshold_otsu  # noqa
from skimage.morphology import ball, binary_erosion, dilation, disk  # noqa
from skimage.segmentation import watershed  # noqa

Let's begin with some median filtering to enhance a bit the edges.
I'm using my own footprint.

And then use Otsu thresolding.

In [ ]:
pixel_resolution = 0.5 / scale["X"]

footprint = np.stack(
    [
        np.zeros_like(disk(pixel_resolution)),
        disk(pixel_resolution),
        np.zeros_like(disk(pixel_resolution)),
    ]
)
midpoint = footprint.shape[1] // 2
footprint[0, midpoint, midpoint] = 1
footprint[2, midpoint, midpoint] = 1

filtered_image = median(img, footprint=footprint)
threshold = threshold_otsu(filtered_image)
nuclei = filtered_image > threshold

In [ ]:
viewer = Viewer()
viewer.add_image(img, scale=spacing)
viewer.add_labels(nuclei, scale=spacing)

Let's apply watershed to get a better separation between nuclei.
We will need the seeds.

In [ ]:
footprint = np.stack([np.zeros_like(disk(1)), disk(1), np.zeros_like(disk(1))])

inter = binary_erosion(nuclei, footprint=footprint)

transformed = distance_transform_edt(inter, sampling=spacing)

distance_between_nuclei = 1.7  # in um
pixel_min_distance = np.floor(distance_between_nuclei / scale["X"]).astype(int)

maxima = peak_local_max(transformed, min_distance=pixel_min_distance)

In [ ]:
print(pixel_min_distance, len(maxima))

We need an image for the edges of the nuclei to limit the watershed.

In [ ]:
edges = scharr(filtered_image)

In [ ]:
viewer = Viewer()
viewer.add_image(transformed, scale=spacing)
viewer.add_image(edges, scale=spacing)
viewer.add_points(
    maxima,
    name="points",
    scale=spacing,
    size=40,
    n_dimensional=True,  # points have 3D "extent"
    face_color="red",
)

In [ ]:
markers = np.zeros(img.shape, dtype=np.uint32)
marker_indices = tuple(np.round(maxima).astype(int).T)
markers[marker_indices] = np.arange(len(maxima)) + 1
markers = dilation(markers, ball(5))

segmented = watershed(
    -transformed,
    markers,
    mask=nuclei,
)

In [ ]:
viewer = Viewer()
viewer.add_image(img, scale=spacing)
viewer.add_labels(segmented, scale=spacing)

It's not the best and should be improved, but it's not that bad.